In [1]:
from datetime import datetime
import pandas as pd
from torch import BoolTensor
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, EarlyStoppingCallback
from transformers import Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


import subprocess
import numpy as np
import random
import torch

import json
import os
import importlib

MAX_LENGTH = 512
DEVICE_DEFAULT = 'cuda'
seed_val = 234
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)


def get_ts():
    return datetime.utcnow().replace(microsecond=0).isoformat()
# end

class SimpleDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item

    def __len__(self):
        return len(self.labels)
    # end
# end

def read_passages(path_data, labels_origin, test_size=0):
    df = pd.read_csv(path_data)

    documents = df['processed'].to_list()
    labels_str = df['target'].to_list()

    samples = documents

    labels_list = sorted(labels_origin)

    labels_all = {l: idx for idx, l in enumerate(labels_list)}

    labels = [labels_all[label_str] for label_str in labels_str]

    if test_size > 0:
        return train_test_split(samples, labels, test_size=test_size, stratify=labels, random_state=234), labels_list
    else:
        return (samples, samples, labels, labels), labels_list
    # end
# end


def compute_metrics(pred):
    labels = pred.label_ids.reshape(-1)
    preds = pred.predictions.argmax(-1).reshape(-1)

    accuracy = accuracy_score(y_true=labels, y_pred=preds)
    precision = precision_score(y_true=labels, y_pred=preds, zero_division=1, average='macro')
    recall = recall_score(y_true=labels, y_pred=preds, zero_division=1, average='macro')
    f1 = f1_score(y_true=labels, y_pred=preds, zero_division=1, average='macro')

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}
# end

def predict_plus(input_tokenized, model):
    masks_sample = input_tokenized.attention_mask

    indicates_sample = BoolTensor(masks_sample == 1)
    indicates_sample = indicates_sample.to(DEVICE_DEFAULT)

    out = model(**input_tokenized.to(DEVICE_DEFAULT), output_attentions=True)

    logits = out.logits.cpu()
    # attentions = out.attentions[-1].cpu()

    # attentions_sum = torch.masked_select((torch.sum(attentions[:, :, 0, :], 1) / attentions.shape[1])[0],
                                         # indicates_sample).tolist()[1:-1]

    # return logits, attentions_sum
    return logits, None
# end


class TrainingEnvironment:
    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer
    # end

    def add_label(self, labels):
        self.labels = labels
        return self
    # end

# end

class ModelFramework_SequenceClassification:
    FILENAME_CONFIG_FRAMEWORK = 'framework.json'
    FILENAME_CHECKSUM = 'checksum.md5'
    FOLDER_DATA = 'data'
    FOLDER_OUTPUT = 'output'

    CONFIG_FRAMEWORK_TEMPLATE = {   # model_full_name = '{model_name}-{model_version}'
        'folder_structure': {
            FOLDER_DATA: '# folder for train/eval data',
            FILENAME_CONFIG_FRAMEWORK: '# jsonfile for framework configuration',
            FILENAME_CHECKSUM: '# framework config checksum',
            FOLDER_OUTPUT: '# folder for output'
        },
        'defaults': {
            'model_name': 'distilbert-base-uncased',
            'model': 'DistilBertForSequenceClassification',
            'tokenizer': 'DistilBertTokenizerFast',
            'tokenizer_name': 'distilbert-base-uncased',
        },
        'model': {
            'model_name': '',
            'model': '',
            'tokenizer': '',
            'labels': [],
            'model_version': '',
            'historical_datasets': []
        }
    }

    def __init__(self, path_folder_base):
        self.path_folder_base = path_folder_base
        self.config_framework = None

        path_file_config_framework = os.path.join(path_folder_base, ModelFramework_SequenceClassification.FILENAME_CONFIG_FRAMEWORK)
        if os.path.exists(path_file_config_framework):
            with open(path_file_config_framework, 'r') as file:
                self.config_framework = json.load(file)
            # end

            print('[INFO] config {} loaded'.format(path_file_config_framework))
        else:
            print('[WARN] please run ModelFramework_SequenceClassification.generate_default_config first.')
        # end

    # end


    @classmethod
    def generate_default_config(cls, path_folder_base):
        os.makedirs(path_folder_base, exist_ok=True)

        path_file_config_framework = os.path.join(path_folder_base, cls.FILENAME_CONFIG_FRAMEWORK)

        with open(path_file_config_framework, 'w+') as file:
            file.write(json.dumps(ModelFramework_SequenceClassification.CONFIG_FRAMEWORK_TEMPLATE))
        # end
    # end


    def load_training_environment(self):
        if not self.config_framework:
            print('[ERROR] initialize failed, config_framework is not loaded')
            return None
        # end

        config_model = self.config_framework['model']

        if not config_model['labels']:
            print('[ERROR] please add labels into framework.config')
            return None
        # end

        if not config_model['model']:
            for k, v in self.config_framework['defaults'].items():  # fill model config by defaults
                config_model[k] = v
            # end
        # end

        model_name = config_model['model_name']
        model_version = config_model['model_version']
        labels = config_model['labels']

        module_transformers = importlib.import_module('transformers')
        klass_model = getattr(module_transformers, config_model['model'])
        klass_tokenizer = getattr(module_transformers, config_model['tokenizer'])


        name_model_full = f"{model_name}-{model_version}" if model_version else model_name
        print('[INFO] loading model: {}'.format(name_model_full))
        model = klass_model.from_pretrained(name_model_full, num_labels=len(labels))
        # model = model.to('cpu')
        
        tokenizer_name = config_model.get('tokenizer_name', model_name)
        print('[INFO] loading tokenizer: {}'.format(tokenizer_name))
        tokenizer = klass_tokenizer.from_pretrained(tokenizer_name, do_lower_case=True)

        env_training = TrainingEnvironment(model, tokenizer).add_label(labels)
        return env_training
    # end

    # update config and save model
    def save_training_environment(self, env_training, version_current):
        model = env_training.model
        config_model = self.config_framework['model']

        name_model = config_model['model_name']
        version_model = config_model['model_version']
        name_model_full = f'{name_model}-{version_model}'
        path_model = os.path.join(self.path_folder_base, name_model_full)
        
        if version_model:
            subprocess.run('rm -rf {}'.format(path_model), shell=True)
            print('[INFO] model {} cleared'.format(name_model_full))
        # end

        name_model_full_current = f'{name_model}-{version_current}'
        path_model_current = os.path.join(self.path_folder_base, name_model_full_current)
        print('[INFO] start to save {}'.format(path_model_current))
        model.save_pretrained(path_model_current)
        print('[INFO] model {} saved'.format(path_model_current))

        config_model['model_version'] = version_current
        config_model['historical_datasets'].append(version_model)

        path_config_framework = os.path.join(self.path_folder_base, ModelFramework_SequenceClassification.FILENAME_CONFIG_FRAMEWORK)
        with open(path_config_framework, 'w+') as file:
            file.write(json.dumps(self.config_framework, indent=4))
        # end
        print('[INFO] config updated to {}'.format(version_current))
    # end

    def train_and_evaluate(self, env_training, path_train, path_test, path_output, path_checkpoint_model):
        print('[INFO] [{}] start main_train_and_evaluate with {} {}'.format(get_ts(), path_train, path_test))

        output_dir = path_checkpoint_model
        labels = env_training.labels
        (train_samples, valid_samples, train_labels, valid_labels), target_names = read_passages(path_train, labels, 0.1)

        tokenizer = env_training.tokenizer
        train_encodings = tokenizer.batch_encode_plus(train_samples, truncation=True, padding=True,
                                                      max_length=MAX_LENGTH,
                                                      return_tensors='pt')
        valid_encodings = tokenizer.batch_encode_plus(valid_samples, truncation=True, padding=True,
                                                      max_length=MAX_LENGTH,
                                                      return_tensors='pt')

        train_dataset = SimpleDataset(train_encodings, train_labels)
        valid_dataset = SimpleDataset(valid_encodings, valid_labels)

        model = env_training.model
        
        training_args = TrainingArguments(
            output_dir=output_dir,  # output directory
            num_train_epochs=8,  # total number of training epochs
            per_device_train_batch_size=16,  # batch size per device during training
            per_device_eval_batch_size=16,  # batch size for evaluation
            warmup_steps=0,  # number of warmup steps for learning rate scheduler
            weight_decay=0.01,  # strength of weight decay
            logging_dir='./logs',  # directory for storing logs
            load_best_model_at_end=True,
            # load the best model when finished training (default metric is loss)    # but you can specify `metric_for_best_model` argument to change to accuracy or other metric
            logging_steps=10,  # log & save weights each logging_steps
            evaluation_strategy="epoch",  # evaluate each `logging_steps`
            learning_rate=2e-5,
            save_strategy='epoch',
            save_total_limit=3,
            metric_for_best_model='f1'
        )

        trainer = Trainer(
            model=model,  # the instantiated Transformers model to be trained
            args=training_args,  # training arguments, defined above
            train_dataset=train_dataset,  # training dataset
            eval_dataset=valid_dataset,  # evaluation dataset
            compute_metrics=compute_metrics,  # the callback that computes metrics of interest
            callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
        )

        # trainer = Trainer(
        #     model=model,  # the instantiated Transformers model to be trained
        #     args=training_args,  # training arguments, defined above
        #     train_dataset=train_dataset,  # training dataset
        #     eval_dataset=valid_dataset,  # evaluation dataset
        #     compute_metrics=compute_metrics
        # )

        print('[INFO] [{}] start training...'.format(get_ts()))
        trainer.train()

        info_state_model = trainer.evaluate()
        print('[INFO] [{}] finish training.'.format(get_ts()))

        ################## start to do eval ##################

        (samples_test, _, indexs_label_test, _), target_names = read_passages(path_test, labels, 0)
        labels_test = [target_names[index_label_test] for index_label_test in indexs_label_test]

        list_conf_output = []
        list_label_output = []
        # list_attention_output = []

        for sample_test, label_origin in zip(samples_test, labels_test):
            input_tokenized = tokenizer.encode_plus(sample_test, padding=True, truncation=True, max_length=MAX_LENGTH,
                                                    return_tensors='pt')
            with torch.no_grad():
                # out = model(**input_tokenized, output_hidden_states=True, output_attentions=True)
                # logits_this, attention_this = predict_plus(input_tokenized, model)
                logits_this, _ = predict_plus(input_tokenized, model)
            # end

            probas_evaluate = torch.nn.functional.softmax(logits_this, dim=-1)
            answer_evaluate = int(probas_evaluate.argmax())
            label_evaluate = target_names[answer_evaluate]

            list_conf_output.append(probas_evaluate.numpy().tolist()[0][answer_evaluate])
            list_label_output.append(label_evaluate)
            # list_attention_output.append(attention_this)
        # end

        print('[INFO] [{}] finish testing.'.format(get_ts()))

        pairs_label_conf = [[a, b, c] for a, b, c in zip(list_label_output, list_conf_output, labels_test)]

        with open(path_output, 'w+') as file:
            file.write(json.dumps(pairs_label_conf))
        # end

        print('[INFO] [{}] main_train_and_evaluate finished.'.format(get_ts()))
    # end

    def main(self, env_training):
        path_folder_checkpoints = 'results'
        path_folder_train = os.path.join(self.path_folder_base, ModelFramework_SequenceClassification.FOLDER_DATA)
        filenames = sorted([filename for filename in os.listdir(path_folder_train) if filename[0] != '.' and 'train' in filename])

        model_last = None

        for _filename in filenames:
            filename_base = _filename.split('_')[0]
            version_current = int(filename_base)    # str_timestamp -> int

            filename_test = f'{filename_base}_test.csv'
            filename_train = _filename
            filename_output = f'output-{filename_base}.json'

            path_data_train = os.path.join(path_folder_train, filename_train)
            path_data_test = os.path.join(path_folder_train, filename_test)
            path_file_output = os.path.join(self.path_folder_base, ModelFramework_SequenceClassification.FOLDER_OUTPUT, filename_output)

            self.train_and_evaluate(env_training, path_data_train, path_data_test, path_file_output, path_folder_checkpoints)
            self.save_training_environment(env_training, version_current)

            subprocess.run(f"rm -rf {path_folder_checkpoints}", shell=True)
        # end
    # end
# end

In [2]:
framework = ModelFramework_SequenceClassification('.')

[INFO] config ./framework.json loaded


In [3]:
env_training = framework.load_training_environment()
framework.main(env_training)

[INFO] loading model: distilbert-base-uncased


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.weight', 'pre_classi

[INFO] loading tokenizer: distilbert-base-uncased
[INFO] [2023-11-22T11:42:29] start main_train_and_evaluate with ./data/202206171000_train_0.35_15.csv ./data/202206171000_test.csv
[INFO] [2023-11-22T11:42:35] start training...


***** Running training *****
  Num examples = 17577
  Num Epochs = 8
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 8792
<ipython-input-1-750f4b9a3411>:38: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.008100,0.038092,0.991295,0.991302,0.991295,0.991256
2,0.001500,0.009769,0.997440,0.997445,0.997440,0.997442
3,0.004900,0.012954,0.997440,0.997485,0.997440,0.997440
4,0.000100,0.005415,0.998464,0.998480,0.998464,0.998466
5,0.000200,0.020478,0.997440,0.997463,0.997440,0.997434
6,0.000000,0.011818,0.997952,0.997954,0.997952,0.997950
7,0.000000,0.005182,0.998464,0.998466,0.998464,0.998464


...

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,2.395400,0.174154,0.916667,0.937500,0.916667,0.914286
2,0.311500,0.000955,1.000000,1.000000,1.000000,1.000000
3,0.002400,0.000205,1.000000,1.000000,1.000000,1.000000
4,0.000400,0.000151,1.000000,1.000000,1.000000,1.000000
5,0.000300,0.000134,1.000000,1.000000,1.000000,1.000000


***** Running Evaluation *****
  Num examples = 24
  Batch size = 16
Saving model checkpoint to results/checkpoint-14
Configuration saved in results/checkpoint-14/config.json
Model weights saved in results/checkpoint-14/pytorch_model.bin
<ipython-input-1-750f4b9a3411>:38: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
***** Running Evaluation *****
  Num examples = 24
  Batch size = 16
Saving model checkpoint to results/checkpoint-28
Configuration saved in results/checkpoint-28/config.json
Model weights saved in results/checkpoint-28/pytorch_model.bin
<ipython-input-1-750f4b9a3411>:38: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceT

[INFO] [2023-11-22T14:52:22] finish training.
[INFO] [2023-11-22T14:52:22] finish testing.
[INFO] [2023-11-22T14:52:22] main_train_and_evaluate finished.
[INFO] model distilbert-base-uncased-202307191009 cleared
[INFO] start to save ./distilbert-base-uncased-202308030456


Configuration saved in ./distilbert-base-uncased-202308030456/config.json
Model weights saved in ./distilbert-base-uncased-202308030456/pytorch_model.bin


[INFO] model ./distilbert-base-uncased-202308030456 saved
[INFO] config updated to 202308030456


In [4]:
print('done')

done
